In [1]:
import pandas as pd
import numpy as np
from faker import Faker
import datetime
fake = Faker()

## Generate the clean dataset

In [2]:
# create EAN8 and EAN13 barcodes with a 00 prefix
def create_ean(data_num=200,ean_len=8, prefix=('00','00'), set_seed=456):
    Faker.seed(set_seed)
    return [fake.ean(prefixes=prefix,length=ean_len) for ean_code in range(0,data_num)]

ean8 = create_ean(ean_len=8)
ean13 = create_ean(ean_len=13)

In [3]:
# create the date column using Faker
date = [fake.date_between(start_date=datetime.date(2018,1,1),
                   end_date=datetime.date(2021, 12, 1))
 for date in range(0,200)]

In [4]:
# create the random generator
rng = np.random.default_rng(2021)

In [5]:
units_sold = rng.integers(low=0, high=100, size=200)

In [6]:
stock = rng.integers(low=0, high=100, size=200)

In [7]:
rating_space = np.arange(0.5,5.0,step=0.1)
rating = rng.choice(rating_space,size=200)

In [8]:
price_space = np.arange(150,10000,step=1)
price_per_unit = rng.choice(price_space,size=200)

In [9]:
# create the cost per product
cost_space = 0.3*price_space
assert round((cost_space/price_space).mean(),1) == 0.3
cost_per_unit = rng.choice(price_space,size=200)

In [10]:
promo = rng.choice(['promo','normal'],size=200)

In [11]:
# create a 25% off promo price
new_price_per_unit = []
for event,price in zip(promo,price_per_unit):
    if event=='promo':
        new_price_per_unit.append(price*.75)
    else:
        new_price_per_unit.append(price)

In [12]:
location = rng.choice(['online','store'],size=200)

In [13]:
color = rng.choice(['black','silver','grey'],size=200)

In [14]:
clean_data = pd.DataFrame({'date':date,
                           'ean8':ean8,
                           'ean13':ean13,
                           'units_sold':units_sold,
                           'stock':stock,
                           'rating':rating,
                           'cost_per_unit':cost_per_unit,
                           'promo':promo,
                           'normal_price': price_per_unit,
                           'new_price_per_unit':new_price_per_unit,
                           'location':location,
                           'color':color
                          })

# make sure the data are in the correct type
clean_data.date = pd.to_datetime(clean_data.date)
clean_data.rating.apply(lambda x: round(x,1))

0      3.7
1      4.9
2      4.1
3      1.6
4      1.3
      ... 
195    3.1
196    2.8
197    2.9
198    4.8
199    1.5
Name: rating, Length: 200, dtype: float64

In [15]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                200 non-null    datetime64[ns]
 1   ean8                200 non-null    object        
 2   ean13               200 non-null    object        
 3   units_sold          200 non-null    int64         
 4   stock               200 non-null    int64         
 5   rating              200 non-null    float64       
 6   cost_per_unit       200 non-null    int64         
 7   promo               200 non-null    object        
 8   normal_price        200 non-null    int64         
 9   new_price_per_unit  200 non-null    float64       
 10  location            200 non-null    object        
 11  color               200 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 18.9+ KB


In [16]:
clean_data.shape

(200, 12)

In [17]:
clean_data.describe()

,units_sold,stock,rating,cost_per_unit,normal_price,new_price_per_unit
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,46.535000,50.440000,2.776000,4759.940000,4968.800000,4378.671250
std,28.433803,27.296187,1.296002,2746.094386,2828.942596,2571.492155
min,0.000000,1.000000,0.500000,158.000000,229.000000,209.250000
25%,23.000000,29.500000,1.600000,2680.500000,2388.000000,2108.125000
50%,45.500000,48.500000,2.800000,4409.500000,5242.500000,4415.375000
75%,71.000000,73.000000,4.000000,7247.750000,7406.750000,6359.250000
max,99.000000,99.000000,4.900000,9894.000000,9927.000000,9927.000000


## Noise insertion process

In [18]:
import copy
dirty_data = copy.deepcopy(clean_data)
dirty_data

,date,ean8,ean13,units_sold,stock,rating,cost_per_unit,promo,normal_price,new_price_per_unit,location,color
0,2021-01-25,00660983,0066098157118,75,79,3.7,3109,normal,8488,8488.00,online,black
1,2020-03-07,00117708,0042232689899,75,68,4.9,7613,normal,5027,5027.00,online,black
2,2021-09-20,00326896,0098054635231,49,27,4.1,1986,normal,5209,5209.00,store,grey
3,2018-04-26,00980548,0050542961673,94,22,1.6,7452,normal,7927,7927.00,store,silver
4,2019-04-18,00237253,0037812373497,66,67,1.3,6949,normal,7652,7652.00,store,grey
...,...,...,...,...,...,...,...,...,...,...,...,...
195,2018-07-28,00415026,0008370264503,50,27,3.1,6900,promo,3781,2835.75,store,grey
196,2020-03-05,00867559,0066214306413,85,62,2.8,2727,promo,3775,2831.25,store,silver
197,2021-03-05,00098403,0006500312049,20,42,2.9,9593,promo,607,455.25,online,black
198,2019-03-30,00032056,0007374372979,24,81,4.8,4621,promo,5632,4224.00,online,grey


In [19]:
# add noise to the promo variable
dirty_data.promo = "normal"

In [20]:
# create a generator and a criteria for selecting when to add noise
rng = np.random.default_rng(124)
criteria = np.arange(0,1.1,step=.1)

In [21]:
# overwrite data
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.5:
          dirty_data.loc[i,'ean8'] = rng.choice(clean_data.ean8,size=1)

In [22]:
# add white space 
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.6:
          dirty_data.loc[i,'location'] = " "+dirty_data.loc[i,'location']+" "

In [23]:
# insert NaNs
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.6:
          dirty_data.loc[i,'color'] = np.nan

In [24]:
# insert out of range data
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.4:
          dirty_data.loc[i,'stock'] = - dirty_data.loc[i,'stock']

In [25]:
# remove the 00 prefix from the EAN13 barcode
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.8:
          dirty_data.loc[i,'ean13'] = int(dirty_data.loc[i,'ean13'])

In [26]:
# duplicate EAN13 barcode data
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.8:
          dirty_data.loc[i,'ean13'] = dirty_data.loc[i,'ean13']*2

In [27]:
# insert out of range data
for i in dirty_data.index:
    if rng.choice(criteria,size=1) > 0.5:
          dirty_data.loc[i,'new_price_per_unit'] = 99999

In [28]:
dirty_data.describe()

,units_sold,stock,rating,cost_per_unit,normal_price,new_price_per_unit
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,46.535000,-4.680000,2.776000,4759.940000,4968.800000,47338.177500
std,28.433803,57.271702,1.296002,2746.094386,2828.942596,47792.626799
min,0.000000,-99.000000,0.500000,158.000000,229.000000,209.250000
25%,23.000000,-51.250000,1.600000,2680.500000,2388.000000,4131.562500
50%,45.500000,-13.500000,2.800000,4409.500000,5242.500000,7789.500000
75%,71.000000,45.250000,4.000000,7247.750000,7406.750000,99999.000000
max,99.000000,99.000000,4.900000,9894.000000,9927.000000,99999.000000


In [29]:
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                200 non-null    datetime64[ns]
 1   ean8                200 non-null    object        
 2   ean13               200 non-null    object        
 3   units_sold          200 non-null    int64         
 4   stock               200 non-null    int64         
 5   rating              200 non-null    float64       
 6   cost_per_unit       200 non-null    int64         
 7   promo               200 non-null    object        
 8   normal_price        200 non-null    int64         
 9   new_price_per_unit  200 non-null    float64       
 10  location            200 non-null    object        
 11  color               116 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 18.9+ KB


In [30]:
dirty_data.to_csv('dirty_data.csv')
clean_data.to_csv('clean_data.csv')